In [7]:
model_name='multi-qa-distilbert-cos-v1'
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

/home/gary/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-07-15 20:29:22.433640: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 20:29:23.287620: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/gary/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the avail

In [2]:
user_question = "I just discovered the course. Can I still join it?"

In [45]:
embedding_model.encode(user_question)[0]

0.078222625

In [46]:
v = embedding_model.encode(user_question)

In [47]:
len(v)

768

In [48]:
scores = X.dot(v)

In [49]:
scores.max()

0.6506573

In [8]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [16]:
documents = [x for x in documents if x['course']=='machine-learning-zoomcamp']

In [17]:
len(documents)

375

In [50]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [18]:
embeddings = []

In [19]:
for doc in documents:
    # print(f"{doc['question']}")

    qa_text = f"{doc['question']} {doc['text']}"

    emb = embedding_model.encode(qa_text)
    embeddings.append(emb)


In [51]:
print(embeddings[0])

[ 8.80590975e-02  1.55936414e-02  7.92557523e-02  2.52757873e-02
  7.55764544e-02 -3.90596576e-02 -4.13813144e-02  2.52917018e-02
  2.43241154e-02  3.62588698e-03 -7.28290575e-03 -3.28750871e-02
  6.12956472e-02 -5.71100302e-02  1.16774915e-02 -1.79441571e-02
  4.49205637e-02 -5.41606732e-02 -1.92254549e-03  1.48329679e-02
  7.91346282e-03 -3.43126804e-02  1.21480739e-02  1.27186067e-02
 -1.60091538e-02  7.12129334e-03  1.58389118e-02 -3.27577436e-04
  3.01294983e-03 -4.58917357e-02 -1.34771829e-03  2.01150198e-02
  2.33724006e-02 -4.28201351e-03 -3.09382826e-02  1.94304902e-03
 -1.88219231e-02  5.35164634e-03  1.29352892e-02 -3.23958993e-02
  3.10556535e-02  5.12984721e-03 -3.24451219e-04 -3.15542296e-02
  1.93376988e-02 -6.38601556e-02 -3.05380933e-02 -6.27885535e-02
 -1.74859986e-02  3.52948830e-02 -2.98071075e-02  7.45346099e-02
 -2.06181873e-02 -1.79156903e-02  1.86262317e-02  5.31014055e-02
  1.19287018e-02 -5.18542863e-02 -1.61929650e-03  3.58705744e-02
 -6.52321987e-03  4.53287

In [9]:
import numpy as np

In [10]:
X = np.array(embeddings)

In [11]:
X.shape

(375, 768)

In [9]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')


In [3]:
# ground_truth

In [4]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
results = search_engine.search(v, num_results=5)


NameError: name 'documents' is not defined

In [30]:
def calculate_hit_rate(search_engine, ground_truth, num_results=5):
    hits = 0
    for item in ground_truth:
        query = item['question']
        expected_document_id = item['document']
        query_embedding = embedding_model.encode(query)
        results = search_engine.search(query_embedding, num_results)
        retrieved_ids = [doc['id'] for doc in results]
        if expected_document_id in retrieved_ids:
            hits += 1
    return hits / len(ground_truth)

hit_rate = calculate_hit_rate(search_engine, ground_truth, num_results=5)
print(hit_rate)

0.9398907103825137


In [29]:
print(ground_truth[:5])

[{'question': 'Where can I sign up for the course?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}, {'question': 'Can you provide a link to sign up?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}, {'question': 'Is there an FAQ for this Machine Learning course?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}, {'question': 'Does this course have a GitHub repository for the sign-up link?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}, {'question': 'How can I structure my questions and answers for the course?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}]


In [33]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '28aaa8f24107', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'AGiapfl4RCq1JdYrVRw9Lg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [52]:
# embeddings

In [64]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "question_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}
        }
    }
}

# Create the index
index_name = "llm-zoomcamp-index"
if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'llm-zoomcamp-index'})

In [62]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872',
 'question_vector': array([ 8.70802850e-02,  5.98304868e-02,  4.62658592e-02, -8.17992017e-02,
         4.93381061e-02,  6.70092329e-02,  5.20505011e-02, -3.99583532e-03,
         2.49936469e-02, -1.05812894e-02, -2.35185027e-03, -3.07780616e-02,
         8.50448459e-02, -3.35008204e-02, -6.30880659e-03, -3.81490290e-02,
         1.95374060e-02,  1.93470959e-02,  1.81184560e-02, -3.73530872e-02,
        -1.95184369e-02,  9.90939140e-03, -1

In [56]:
# for doc in tqdm(documents):
#     question = doc['question']
#     text = doc['text']
#     qt = question + ' ' + text

#     doc['question_vector'] = embedding_model.encode(question)
#     doc['text_vector'] = embedding_model.encode(text)
#     doc['question_text_vector'] = embedding_model.encode(qt)

100%|██████████| 375/375 [00:06<00:00, 57.18it/s]


In [65]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:03<00:00, 109.20it/s]


In [72]:
def calculate_hit_rate(search_engine, ground_truth, num_results=5):
    hits = 0
    for item in ground_truth:
        query = item['question']
        expected_document_id = item['document'] 
        query_embedding = embedding_model.encode(query)
        results = search_engine.search(query_embedding, num_results)
        retrieved_ids = [doc['id'] for doc in results]
        if expected_document_id in retrieved_ids:
            hits += 1
    return hits / len(ground_truth)

hit_rate = calculate_hit_rate(search_engine, ground_truth, num_results=5)
print(hit_rate)

0.9398907103825137


In [85]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [86]:
# search_query = {
#     "field": "question_vector",
#     "query_vector": v,
#     "k": 5,
#     "num_candidates": 1000,
# }

# res = es_client.search(index=index_name, knn=search_query, source=['text', 'section', 'question', 'course', 'id'])


In [87]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = embedding_model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

In [88]:
response = question_vector_knn({"question":"I just discovered the course. Can I still join it?","course":"machine-learning-zoomcamp"})

In [89]:
for hit in response['hits']['hits']:
    print(hit)

TypeError: list indices must be integers or slices, not str

In [82]:
response['hists']

TypeError: list indices must be integers or slices, not str

In [90]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [92]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
    }

In [93]:
print(evaluate(ground_truth, question_vector_knn))

100%|██████████| 1830/1830 [00:44<00:00, 40.80it/s]


{'hit_rate': 0.8076502732240437}

In [94]:
### Trying question 6 again ###

In [10]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [14]:
# documents[0]

In [11]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [12]:
documents = [x for x in documents if x['course']=='machine-learning-zoomcamp']

In [13]:
index_name = "course-questions"

In [15]:
from tqdm import tqdm
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = embedding_model.encode(question)
    doc['text_vector'] = embedding_model.encode(text)
    doc['question_text_vector'] = embedding_model.encode(qt)

    es_client.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:53<00:00,  7.01it/s]


In [44]:
v = embedding_model.encode("I just discovered the course. Can I still join it?")

In [45]:
index_name

'course-questions'

In [50]:
search_query = {
    "field": "question_vector",
    "query_vector": v,
    "k": 5,
    "num_candidates": 1000,
}

es_results = es_client.search(
    index=index_name, 
    knn=search_query, 
    source=['text', 'section', 'question', 'course', 'id'])

In [57]:
es_results['hits']

False

In [73]:
for i in ground_truth:
    if i['course'] != 'machine-learning-zoomcamp':
        print(i)

In [16]:
def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": 'machine-learning-zoomcamp'
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [20]:
def question_vector_knn(q):
    cnt = 0
    for i in q:
        question = i['question']
        # course = q['course']

        v_q = embedding_model.encode(question)

        if i['document'] in [x['id'] for x in elastic_search_knn('question_text_vector',embedding_model.encode('Where can I sign up for the course?'))]:
            cnt += 1
    return cnt / len(documents)

In [21]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [22]:
print(question_vector_knn(ground_truth))

0.06666666666666667


In [65]:
elastic_search_knn('question_vector', embedding_model.encode('Where can I sign up for the course?'), 'machine-learning-zoomcamp')

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

100%|██████████| 8499999/8499999 [00:01<00:00, 6843804.20it/s]
